In [ ]:
import pandas as pd
import numpy as np
import re

#1️ Đọc dữ liệu từ file CSV
file_path = "patient_heart_rate.csv"
df = pd.read_csv(file_path, header=None)

In [ ]:
#2️ Đặt tên cột cho dataset (giả sử đúng thứ tự)
df.columns = ["Name", "Age", "Weight", "HR1", "HR2", "HR3", "HR4", "HR5", "HR6"]

In [ ]:
#3️ Tách cột Name thành Firstname và Lastname
df[['Firstname', 'Lastname']] = df['Name'].str.split(expand=True)
df.drop(columns=['Name'], inplace=True)

In [ ]:
#4️ Chuyển đổi đơn vị Weight về kg
def convert_weight(value):
    if "lbs" in str(value):
        return round(float(str(value).replace("lbs", "")) * 0.453592, 2)  # lbs -> kg
    return float(str(value).replace("kg", ""))
df["Weight"] = df["Weight"].apply(convert_weight)

In [ ]:
#5️ Xóa dòng trống và dòng trùng lặp
df.dropna(how="all", inplace=True)
df.drop_duplicates(inplace=True)

In [ ]:
#6️ Xử lý ký tự không phải ASCII
def remove_non_ascii(text):
    return re.sub(r'[^\x00-\x7F]+', '', str(text))
df["Firstname"] = df["Firstname"].apply(remove_non_ascii)
df["Lastname"] = df["Lastname"].apply(remove_non_ascii)

In [ ]:
#7️ Xử lý dữ liệu bị thiếu
age_mean = df["Age"].mean()
df["Age"].fillna(age_mean, inplace=True)

weight_mean = df["Weight"].mean()
df["Weight"].fillna(weight_mean, inplace=True)

df.dropna(subset=["Age", "Weight"], how="all", inplace=True)


In [ ]:
#8️ Xử lý giá trị thiếu trên HR5
def fill_missing_hr(row, col):
    hr_cols = ["HR1", "HR2", "HR3", "HR4", "HR5", "HR6"]
    idx = hr_cols.index(col)
    prev_value = row[hr_cols[idx - 1]] if idx > 0 else np.nan
    next_value = row[hr_cols[idx + 1]] if idx < len(hr_cols) - 1 else np.nan
    if pd.notna(prev_value) and pd.notna(next_value):
        return (prev_value + next_value) / 2
    elif pd.notna(prev_value):
        return prev_value
    elif pd.notna(next_value):
        return next_value
    elif pd.notna(row[hr_cols].mean()):
        return row[hr_cols].mean()
    else:
        return df[hr_cols].mean().mean()
df["HR5"] = df.apply(lambda row: fill_missing_hr(row, "HR5"), axis=1)


In [ ]:
# Reindex lại dữ liệu
df.reset_index(drop=True, inplace=True)

df.head()
